In [1]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
from unidecode import unidecode
from fuzzywuzzy import process

import os
import zipfile
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
import fiona
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

c:\Users\Administrador\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
df_matricula_R1 = pd.read_csv("input/20230526_R1_Matricula.csv")
df_matricula_R1_v2 = df_matricula_R1[df_matricula_R1["MATRICULA_INMOBILIARIA"].notnull()][["NPN", "MATRICULA_INMOBILIARIA", "NUMERO_DEL_PREDIO", 'Departamento', 'Municipio', "DEPARTAMENTO", "DIVIPOLA"]]
df_matricula_R1_v2 = df_matricula_R1_v2.rename(columns={"NPN" : "NPN_R1", "MATRICULA_INMOBILIARIA" : "MATRICULA_INMOBILIARIA_R1", "NUMERO_DEL_PREDIO" : "NUMERO_DEL_PREDIO_R1", 'Departamento' : 'Departamento_R1', 'Municipio' : 'Municipio_R1', "DEPARTAMENTO" : "CODIGO_DEPARTAMENTO_R1", "DIVIPOLA" : "DIVIPOLA_R1"})
df_matricula_R1_v2.columns
#df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1_v2"] = df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"].str.strip()
df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"] = df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"].str.strip()
pattern = r'[^a-zA-Z0-9]'

#df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1_v2'] = df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1_v2'].astype(str).str.replace(pattern, '', regex = True)

C:\Users\Administrador\AppData\Local\Temp\ipykernel_19812\2556328726.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_matricula_R1 = pd.read_csv("input/20230526_R1_Matricula.csv")


In [5]:
df_matricula_R1_v3 = df_matricula_R1_v2[df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1'].str.count("-") == 1]
df_matricula_R1_v3['Matricula'] = df_matricula_R1_v3['MATRICULA_INMOBILIARIA_R1'].str.extract(r'-(\d+)')
df_matricula_R1_v3['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_matricula_R1_v3['DIVIPOLA_R1'].astype(int).apply(round).astype(str).str.strip() + "-" + df_matricula_R1_v3['Matricula'].astype(str).str.strip()

C:\Users\Administrador\AppData\Local\Temp\ipykernel_19812\3602553612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matricula_R1_v3['Matricula'] = df_matricula_R1_v3['MATRICULA_INMOBILIARIA_R1'].str.extract(r'-(\d+)')
C:\Users\Administrador\AppData\Local\Temp\ipykernel_19812\3602553612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matricula_R1_v3['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_matricula_R1_v3['DIVIPOLA_R1'].astype(int).apply(round).astype(str).str.strip() + "-" + df_matricula_R1

In [6]:
ofertas_final = pd.read_csv("output/230626_RESULTADOS_GEOREF.csv")

C:\Users\Administrador\AppData\Local\Temp\ipykernel_19812\1804148341.py:1: DtypeWarning: Columns (4,5,12,13,14,16,18,19,20,21,22,23,24,25,26,31,32,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  ofertas_final = pd.read_csv("output/230626_RESULTADOS_GEOREF.csv")


In [8]:
df_R1_matricula = pd.read_csv("../230628_COMPARACION_R1_DIFERENTES_FUENTES/output/230703_R1_Matricula.csv", encoding = "latin1")

C:\Users\Administrador\AppData\Local\Temp\ipykernel_19812\2950384574.py:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_R1_matricula = pd.read_csv("../230628_COMPARACION_R1_DIFERENTES_FUENTES/output/230703_R1_Matricula.csv", encoding = "latin1")


MemoryError: Unable to allocate 133. MiB for an array with shape (8736232,) and data type complex128

In [166]:
df_matriculas = pd.DataFrame({"Matricula_Inmobiliaria" : ofertas_final["Matricula_Inmobiliaria"].str.strip().str.extractall(r'(\d+)')[0].reset_index(level=1, drop=True)})
df_matriculas.tail()

df_merge = pd.merge(df_matriculas, ofertas_final[["DIVIPOLA"]], how = "left", left_index = True, right_index = True)
#ofertas_final[["DIVIPOLA"]]

In [167]:
df_merge["DIVIPOLA"] = df_merge["DIVIPOLA"].fillna(0).astype(int)
df_merge["Matricula_Divipola"] = df_merge["DIVIPOLA"].apply(round).astype(str) + "-" + df_merge["Matricula_Inmobiliaria"].astype(str)
df_merge.tail(15)

,Matricula_Inmobiliaria,DIVIPOLA,Matricula_Divipola
503478,52871,25001,25001-52871
503479,408782,25001,25001-408782
503480,18795,8832,8832-18795
503481,712956,76364,76364-712956
503482,46854,23162,23162-46854
503483,43696,23162,23162-43696
503484,248788,25001,25001-248788
503485,785092,76001,76001-785092
503486,20134774,25001,25001-20134774
503487,22210,76001,76001-22210


In [168]:
df_matricula_R1_v3['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_matricula_R1_v3['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'].str.strip().str.replace(" ", "").astype(str)
df_merge['Matricula_Divipola'] = df_merge['Matricula_Divipola'].str.strip().str.replace(" ", "").astype(str)
df_merge.reset_index(inplace= True)
df_merge_2 = pd.merge(df_merge, df_matricula_R1_v3, how = "inner", left_on = 'Matricula_Divipola', right_on = 'MATRICULA_INMOBILIARIA_DIVIPOLA_R1')
df_merge_2.drop_duplicates("index",inplace= True)
df_merge_2.set_index("index", inplace= True)

C:\Users\Administrador\AppData\Local\Temp\ipykernel_18880\3381431116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matricula_R1_v3['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_matricula_R1_v3['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'].str.strip().str.replace(" ", "").astype(str)


In [187]:
#Matricula_Inmobiliaria	DIVIPOLA	Matricula_Divipola
#ofertas_final_v2.columns
ofertas_final_v2[ofertas_final_v2["NPN_R1"].notnull()][['Codigo_ORIP', "Matricula_Inmobiliaria_x","Matricula_Inmobiliaria_y","DIVIPOLA_y","Matricula_Divipola"]]

,Codigo_ORIP,Matricula_Inmobiliaria_x,Matricula_Inmobiliaria_y,DIVIPOLA_y,Matricula_Divipola
21,307,98771,98771,25612.000000,25612-98771
36,152,74617,74617,25181.000000,25181-74617
70,176,193387,193387,25817.000000,25817-193387
82,176,190660,190660,25817.000000,25817-190660
84,176,201456,201456,25817.000000,25817-201456
...,...,...,...,...,...
503474,140,65113,65113,23001.000000,23001-65113
503475,143,49680,49680,23001.000000,23001-49680
503480,´040,18795,18795,8832.000000,8832-18795
503482,143,46854,46854,23162.000000,23162-46854


In [169]:
#print(ofertas_final.tail())
df_merge_2["FUENTE"] = "R1_MATRICULA"
#print(df_merge_2.tail())
ofertas_final_v2 = pd.merge(ofertas_final, df_merge_2, left_index = True, right_index = True, how = "left")

In [170]:
quantiles = list(np.arange(0, 0.95, 0.05))#[0.05, 0.1, 0.2, 0.25, 0.50, 0.75, 0.90, 0.95, 0.97]
ofertas_final_v2[(ofertas_final_v2["ZONA"] == "RURAL_APROX") & (ofertas_final_v2["FUENTE"] == "R1_MATRICULA")]["Distancia_final"].quantile(quantiles).to_frame().reset_index()



,index,Distancia_final
0,0.000000,0.000003
1,0.050000,0.000032
2,0.100000,0.000082
3,0.150000,0.002409
4,0.200000,0.006006
5,0.250000,0.006008
6,0.300000,0.006335
7,0.350000,0.006340
8,0.400000,0.006340
9,0.450000,0.006340


In [171]:
pd.options.display.float_format = '{:.6f}'.format
ofertas_final_v2[(ofertas_final_v2["ZONA"] == "URBANA_APROX") & (ofertas_final_v2["FUENTE"] == "R1_MATRICULA")]["Distancia_final"].quantile(quantiles).to_frame().reset_index()

,index,Distancia_final
0,0.000000,0.000000
1,0.050000,0.000003
2,0.100000,0.000006
3,0.150000,0.000008
4,0.200000,0.000014
5,0.250000,0.000020
6,0.300000,0.000026
7,0.350000,0.000030
8,0.400000,0.000037
9,0.450000,0.000043


In [177]:
ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "CODIGO_PREDIAL"] = ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "NPN_R1"]
#ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "Fuente"] = "OIC"
ofertas_final_v2.loc[(ofertas_final_v2["NPN_R1"].isnull()) & (ofertas_final_v2["CODIGO_PREDIAL"].notnull()), "FUENTE"] = "Subdireccion"

ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.000092) & (ofertas_final_v2["ZONA"] == "URBANA_APROX"), "TIPO_GEORREF"] = "APROX_OK"
ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.008925) & (ofertas_final_v2["ZONA"] == "RURAL_APPROX"), "TIPO_GEORREF"] = "APROX_OK"


In [178]:
ofertas_final_v2[["TIPO_GEORREF", "FUENTE"]].value_counts(dropna = False).to_frame()

count
TIPO_GEORREF FUENTE              
APROX        Subdireccion  268579
NaN          NaN           163004
APROX_OK     Subdireccion   29655
EXACTA       Subdireccion   18289
             R1_MATRICULA    9672
APROX_OK     R1_MATRICULA    8715
APROX        R1_MATRICULA    5426
EXACTA       NaN               82
NaN          R1_MATRICULA      71

In [179]:
ofertas_final_v2[["TIPO_GEORREF", "FUENTE", "ZONA"]].value_counts(dropna = False).to_frame()

count
TIPO_GEORREF FUENTE       ZONA                
APROX        Subdireccion RURAL_APROX   207933
NaN          NaN          NaN           163004
APROX        Subdireccion URBANA_APROX   60646
APROX_OK     Subdireccion URBANA_APROX   29655
EXACTA       Subdireccion URBANO         16692
             R1_MATRICULA URBANO          8788
APROX_OK     R1_MATRICULA URBANA_APROX    8715
APROX        R1_MATRICULA URBANA_APROX    2928
                          RURAL_APROX     2498
EXACTA       Subdireccion RURAL           1597
             R1_MATRICULA RURAL            766
                          NaN              118
             NaN          NaN               82
NaN          R1_MATRICULA NaN               71

In [175]:
ofertas_final_v2.drop((['Confidence_arcgis',  'address', 'CODIGO_ANT_URBANO', 'CODIGO_ANT_RURAL_APROX', 'CODIGO_ANT_URBANO_APROX']), axis = 1, inplace = True)
ofertas_final_v2.to_csv("output/230626_BASE_PROCESADA_AVALUOS_IGAC.csv")

In [147]:
df_cruce_ofertas_R1[df_cruce_ofertas_R1["ZONA"] == "RURAL_APROX"]["Distancia_final"].quantile(quantiles)



0.05     0.000052
0.10     0.004176
0.20     0.006190
0.25     0.008511
0.50     0.035985
0.75     0.064048
0.90    24.048988
0.95    28.122124
Name: Distancia_final, dtype: float64

In [148]:
df_cruce_ofertas_R1[df_cruce_ofertas_R1["ZONA"] == "URBANA_APROX"]["Distancia_final"].quantile(quantiles)

0.05    0.000005
0.10    0.000009
0.20    0.000019
0.25    0.000023
0.50    0.000043
0.75    0.000074
0.90    0.003017
0.95    0.006631
Name: Distancia_final, dtype: float64

In [165]:
df_cruce_ofertas_R1[["TIPO_GEORREF", "Fuente"]].value_counts(dropna = False).to_frame()

count
TIPO_GEORREF Fuente              
APROX        Subdireccion  242928
NaN          NaN           186099
APROX_OK     Subdireccion   41139
EXACTA       Subdireccion   22083
             OIC             5391
APROX_OK     OIC             4136
NaN          OIC             2757
APROX        OIC             1225
EXACTA       NaN               96

In [166]:
df_cruce_ofertas_R1[["TIPO_GEORREF", "Fuente", "ZONA"]].value_counts(dropna = False).to_frame()

count
TIPO_GEORREF Fuente       ZONA                
APROX        Subdireccion RURAL_APROX   191233
NaN          NaN          NaN           186099
APROX        Subdireccion URBANA_APROX   51695
APROX_OK     Subdireccion URBANA_APROX   41139
EXACTA       Subdireccion URBANO         20118
             OIC          URBANO          5164
APROX_OK     OIC          URBANA_APROX    4136
NaN          OIC          NaN             2757
EXACTA       Subdireccion RURAL           1965
APROX        OIC          RURAL_APROX      765
                          URBANA_APROX     460
EXACTA       OIC          RURAL            130
                          NaN               97
             NaN          NaN               96